In [1]:
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import glob
import copy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision
from torchvision import transforms, models 

from efficientnet_pytorch import EfficientNet


# 데이터 확인 및 데이터 정리

In [2]:
def get_image_paths(directory, file_list):
    image_paths = []
    for folder_name in file_list:
        folder_path = os.path.join(directory, folder_name)
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.jpg'):  # .jpg 파일만 선택
                image_paths.append(os.path.join(folder_path, file_name))
    return image_paths

In [3]:
#데이터 위치 (train,val, test까지만)
data_dir = '/home/hts/A_project/hts_pytorch/data/birds_image_data/'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

#train,val, test안에 class확인
train_list = sorted(os.listdir(train_dir))
val_list = sorted(os.listdir(val_dir))
test_list = sorted(os.listdir(test_dir))

#모든 폴더안에 위치 가져오기
train_images = get_image_paths(train_dir, train_list)
val_images = get_image_paths(val_dir, val_list)
test_images = get_image_paths(test_dir, test_list)

#클래스 인코딩 변수
class_to_int = {train_list[i] : i for i in range(len(train_list))}

# 클래스 및 데이터 개수 출력
print(f"(train class) : {len(train_list)}개 {train_list}")
print(f"    총 데이터  : {len(train_images)}")
print(f"(val class)   : {len(val_list)}개 {val_list}")
print(f"    총 데이터  : {len(val_images)}")
print(f"(test class)  : {len(test_list)}개 {test_list}")
print(f"    총 데이터  : {len(test_images)}")

(train class) : 525개 ['ABBOTTS BABBLER', 'ABBOTTS BOOBY', 'ABYSSINIAN GROUND HORNBILL', 'AFRICAN CROWNED CRANE', 'AFRICAN EMERALD CUCKOO', 'AFRICAN FIREFINCH', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN PYGMY GOOSE', 'ALBATROSS', 'ALBERTS TOWHEE', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'ALTAMIRA YELLOWTHROAT', 'AMERICAN AVOCET', 'AMERICAN BITTERN', 'AMERICAN COOT', 'AMERICAN DIPPER', 'AMERICAN FLAMINGO', 'AMERICAN GOLDFINCH', 'AMERICAN KESTREL', 'AMERICAN PIPIT', 'AMERICAN REDSTART', 'AMERICAN ROBIN', 'AMERICAN WIGEON', 'AMETHYST WOODSTAR', 'ANDEAN GOOSE', 'ANDEAN LAPWING', 'ANDEAN SISKIN', 'ANHINGA', 'ANIANIAU', 'ANNAS HUMMINGBIRD', 'ANTBIRD', 'ANTILLEAN EUPHONIA', 'APAPANE', 'APOSTLEBIRD', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASHY THRUSHBIRD', 'ASIAN CRESTED IBIS', 'ASIAN DOLLARD BIRD', 'ASIAN GREEN BEE EATER', 'ASIAN OPENBILL STORK', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AUSTRALASIAN FIGBIRD', 'AVADAVAT', 'AZARAS SPINETAIL', 'AZURE BREASTED PITTA', 'AZURE JAY

# 데이터셋 제작

In [4]:
class biard_data_set():
    def __init__(self, images, class_to_int, transformer=None):
        self.images = images
        self.class_to_int = class_to_int
        self.transformer = transformer

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_path = self.images[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = image_path.split("/")[-2]
        label = self.class_to_int[label]

        if self.transformer:
            image = self.transformer(image)

        return image, label

In [5]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [6]:
def build_dataloader(T_batch_size=200, V_batch_size=128):
    dataloader = {}

    train_set = biard_data_set(images=train_images, class_to_int=class_to_int, transformer=transformer)
    val_set = biard_data_set(images=val_images, class_to_int=class_to_int, transformer=transformer)

    dataloader['train'] = DataLoader(dataset=train_set, batch_size=T_batch_size, num_workers=4, shuffle=True)
    dataloader['val'] = DataLoader(dataset=val_set, batch_size=V_batch_size, num_workers=4, shuffle=False)

    return dataloader

# 모델 제작

In [7]:
class EfficientNet_class(nn.Module):
    def __init__(self):
        super(EfficientNet_class, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=525)
        # for n, p in self.model.named_parameters():
        #     if '_fc' not in n:
        #         p.requires_grad = False

    def forward(self, x):
        x = self.model(x)

        return x

In [8]:
model = EfficientNet_class()
print(model(torch.randn(1,3,224,224)).shape)

Loaded pretrained weights for efficientnet-b0
torch.Size([1, 525])


# 학습

In [9]:
def train_one_epoch(dataloaders, model, optimizer, device):
    total_loss = {}

    for phase in ['train', 'val']:
        running_loss = 0.0

        if phase == 'train':
            model.train()
        elif phase == 'val':
            model.eval()
        with tqdm(dataloaders[phase], unit='batch', desc=f'Epoch {phase}') as tepoch:
            for batch in tepoch:

                image = batch[0].to(device)
                target = batch[1].to(device)

                with torch.set_grad_enabled(phase =='train'):
                    predict = model(image)

                    loss = F.cross_entropy(predict, target, reduction = 'mean')

                    if phase =='train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item()
                tepoch.set_postfix(loss = running_loss/ (tepoch.n +1))
        total_loss[phase] = running_loss/len(dataloaders[phase])
    return total_loss

In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
dataloaders = build_dataloader()
model = EfficientNet_class().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
num_epoch = 300
train_loss = []
val_loss = []
best_loss = 100

for epoch in range(num_epoch):
    loss = train_one_epoch(dataloaders=dataloaders, model=model, optimizer=optimizer, device=device)
    train_loss.append(loss['train'])
    val_loss.append(loss['val'])
    print(f"{epoch+1}/{num_epoch}--train_loss : {loss['train']}, val_loss : {loss['val']}")

    if(loss['val']<best_loss):
        best_loss = loss['val']
        best_model = copy.deepcopy(model.state_dict())
        os.makedirs('./trained_model/', exist_ok=True)
        torch.save(best_model, os.path.join('./trained_model/','bestmodel.pt'),_use_new_zipfile_serialization=False)

print(f'bestmodel : {best_loss}')

Loaded pretrained weights for efficientnet-b0


Epoch train:   4%|▍         | 17/424 [00:07<02:50,  2.39batch/s, loss=6.25]